In [11]:
import pandas as pd

In [ ]:
# Read and combine files
p20171 = pd.read_csv("./data/ICFES/SB11-20171-RGSTRO-CLFCCN-V1-0.TXT", sep="¬", engine="python")

p20172 = pd.read_csv("./data/ICFES/SB11-20172-RGSTRO-CLFCCN-V1-0.TXT", sep="¬", engine="python")

In [54]:
#icfes = p20171.append(p20172, ignore_index=True)
icfes = pd.concat([p20171, p20172], ignore_index=True)
icfes.head()
icfes.dtypes

COLE_AREA_UBICACION               object
COLE_BILINGUE                     object
COLE_CALENDARIO                   object
COLE_CARACTER                     object
COLE_CODIGO_ICFES                  int64
COLE_COD_DANE_ESTABLECIMIENTO      int64
COLE_COD_DANE_SEDE                 int64
COLE_COD_DEPTO_UBICACION           int64
COLE_COD_MCPIO_UBICACION           int64
COLE_DEPTO_UBICACION              object
COLE_GENERO                       object
COLE_JORNADA                      object
COLE_MCPIO_UBICACION              object
COLE_NATURALEZA                   object
COLE_NOMBRE_ESTABLECIMIENTO       object
COLE_NOMBRE_SEDE                  object
COLE_SEDE_PRINCIPAL               object
DESEMP_C_NATURALES                 int64
DESEMP_INGLES                     object
DESEMP_LECTURA_CRITICA             int64
DESEMP_MATEMATICAS                 int64
DESEMP_SOCIALES_CIUDADANAS         int64
ESTU_COD_DEPTO_PRESENTACION        int64
ESTU_COD_MCPIO_PRESENTACION        int64
ESTU_COD_RESIDE_

In [67]:
icfes.loc[13529:, icfes.dtypes == object]['COLE_NOMBRE_ESTABLECIMIENTO']
# Replace extrange characters in string values
icfes.replace(to_replace=('\"', '`', '\''), value=('', ' ', ' '), regex=True, inplace=True)
icfes[icfes['COLE_NOMBRE_ESTABLECIMIENTO'].str.contains("IDEBIC")].iloc[0:5, 14:]
#icfes.loc[[13529]].str.replace('\"','')

,COLE_NOMBRE_ESTABLECIMIENTO,COLE_NOMBRE_SEDE,COLE_SEDE_PRINCIPAL,DESEMP_C_NATURALES,DESEMP_INGLES,DESEMP_LECTURA_CRITICA,DESEMP_MATEMATICAS,DESEMP_SOCIALES_CIUDADANAS,ESTU_COD_DEPTO_PRESENTACION,ESTU_COD_MCPIO_PRESENTACION,...,PERCENTIL_LECTURA_CRITICA,PERCENTIL_MATEMATICAS,PERCENTIL_SOCIALES_CIUDADANAS,PERIODO,PUNT_C_NATURALES,PUNT_GLOBAL,PUNT_INGLES,PUNT_LECTURA_CRITICA,PUNT_MATEMATICAS,PUNT_SOCIALES_CIUDADANAS
17826,IE IDEBIC,KWE SX NASA KSXA WNXI,S,3,A-,3,3,3,76,76275,...,78,60,94,20172,62,296,38,61,53,68
25793,IE IDEBIC,KWE SX NASA KSXA WNXI,S,2,A-,2,2,2,76,76275,...,20,32,24,20172,52,226,41,44,44,42
42970,IE IDEBIC,KWE SX NASA KSXA WNXI,S,2,A-,3,2,3,76,76275,...,57,45,69,20172,53,260,41,55,48,56
42973,IE IDEBIC,KWE SX NASA KSXA WNXI,S,2,A-,2,2,1,76,76275,...,25,26,7,20172,45,207,38,46,42,34
43733,IE IDEBIC,KWE SX NASA KSXA WNXI,S,3,A-,4,3,3,76,76275,...,90,91,89,20172,69,324,47,66,66,64


In [68]:
icfes.to_csv("./data/ICFES/Todo.csv", index=False, encoding="utf-8")